In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pydataset import data
import statistics
import seaborn as sns
import env
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import scipy
from scipy import stats
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model
import sklearn.preprocessing
import warnings
warnings.filterwarnings("ignore")
# importing my personal wrangle module
import wrangle
import model
import viz

In [ ]:
# Acquiring the data using the wrangle
zillow_sql_query = wrangle.get_zillow_data()
# I am working with a copy of the SQL query to be able to access it faster
zillow = zillow_sql_query.copy()

In [ ]:
zillow = wrangle.clean_and_prep_data(zillow)

In [ ]:
encoded = wrangle.encode_zillow(zillow)

In [ ]:
train, validate, test,X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.split_zillow(encoded)

In [ ]:
train_scaled, X_train_scaled, y_train_scaled, \
validate_scaled, X_validate_scaled, y_validate_scaled, \
test_scaled, X_test_scaled, y_test_scaled = wrangle.scale_zillow(train, validate, test)

In [ ]:
X_train_kbest, X_validate_kbest, X_test_kbest, X_train_rfe, X_validate_rfe, X_test_rfe \
= model.create_data_for_models(X_train_scaled, X_validate_scaled, X_test_scaled)

In [ ]:
# 1. Predict tax_value_pred_mean
tax_value_pred_mean = y_train_scaled.tax_value.mean()
y_train_scaled['tax_value_pred_mean'] = round(tax_value_pred_mean, 2)
y_validate_scaled['tax_value_pred_mean'] = round(tax_value_pred_mean,2)
y_test_scaled['tax_value_pred_mean'] = round(tax_value_pred_mean,2)

In [ ]:

# 3. RMSE of tax_value_pred_mean
from sklearn.metrics import mean_squared_error


rmse_train = mean_squared_error(y_train_scaled.tax_value,
                                y_train_scaled.tax_value_pred_mean) ** .5
rmse_validate = mean_squared_error(y_validate_scaled.tax_value, y_validate_scaled.tax_value_pred_mean) ** (0.5)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

In [ ]:
viz.actual_vs_predicted(y_train_scaled)

In [ ]:
metric_df = pd.DataFrame(data=[
            {
                'model': 'mean_baseline', 
                'RMSE_train': rmse_train,
                'RMSE_validate': rmse_validate
                }
            ])
metric_df

In [ ]:
metric_df = model.run_ols_model_kbest(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, metric_df)


In [ ]:
metric_df = model.run_ols_model_rfe(X_train_rfe, y_train_scaled, X_validate_rfe, y_validate_scaled, metric_df)


In [ ]:
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

metric_df = model.lasso_lars_kbest(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, metric_df)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate,
     "\nDifference: ", rmse_validate - rmse_train)

In [ ]:
metric_df = model.lasso_lars_rfe(X_train_rfe, y_train_scaled, X_validate_rfe, y_validate_scaled, metric_df)

In [ ]:
metric_df = model.tweedie_kbest(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, metric_df)

In [ ]:
metric_df = model.tweedie_rfe(X_train_rfe, y_train_scaled, X_validate_rfe, y_validate_scaled, metric_df)


In [ ]:
metric_df = model.polynomial_regression_kbest(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, \
                                              X_test_kbest, metric_df)

In [ ]:
metric_df = model.polynomial_regression_rfe(X_train_rfe, y_train_scaled, X_validate_rfe, y_validate_scaled, X_test_rfe, metric_df)


In [ ]:
metric_df

In [ ]:
metric_df = model.run_all_models_on_all_data(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, X_train_rfe, X_validate_rfe, X_test_kbest, X_test_rfe, metric_df)


In [ ]:
metric_df

In [ ]:
viz.age_by_county(train)